In [1]:
import pandas as pd
import numpy as np

In [2]:
wdi_wide = pd.read_csv("../data/wdi.csv")

In [3]:
wdi_wide = wdi_wide.drop(columns=["Country Name", "Country Code", "Series Code"])

In [4]:
year_cols = [col for col in wdi_wide.columns if len(col) >= 4 and col[:4].isdigit()]

In [5]:
wdi_long = wdi_wide.melt(
    id_vars=['Series Name'],
    value_vars=year_cols,
    var_name='Year',
    value_name='Value'
)


In [6]:
wdi_long['Year'] = wdi_long['Year'].apply(lambda x: x.split()[0])
wdi_long['Year'] = pd.to_numeric(wdi_long['Year'], errors='coerce')

In [7]:
wdi_long['Value'] = pd.to_numeric(wdi_long['Value'], errors='coerce')

In [8]:
wdi_pivot = wdi_long.pivot_table(
    index='Year',
    columns='Series Name',
    values='Value'
).reset_index()

In [9]:
wdi_pivot.columns.name = None

In [10]:
wdi_pivot = wdi_pivot.reset_index(drop=True)

In [11]:
wdi_pivot = wdi_pivot.dropna()

In [12]:
fred = pd.read_csv("../data/fred.csv")

In [13]:
fred.columns = ["Date", "Consumer confidence"]

In [14]:
fred['Year'] = pd.to_datetime(fred['Date']).dt.year

In [15]:
fred_annual = fred.groupby('Year')['Consumer confidence'].mean().reset_index()

In [16]:
df = wdi_pivot.merge(fred_annual, on='Year', how='left')

In [17]:
df = df.dropna()

In [18]:
df.to_csv("../data/cleaned.csv", index=False)